## Loading dataset 

In [1]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [2]:
! kaggle datasets download -d rahilmehtaucoe2784/fashion

 98% 186M/191M [00:02<00:00, 88.4MB/s]
100% 191M/191M [00:02<00:00, 76.3MB/s]


In [3]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

## installing and importing Libraries 

In [4]:
!pip install -q tflite-model-maker
!pip install -q pycocotools

     |████████████████████████████████| 616 kB 14.3 MB/s 
     |████████████████████████████████| 120 kB 68.8 MB/s 
     |████████████████████████████████| 234 kB 67.9 MB/s 
     |████████████████████████████████| 87 kB 8.9 MB/s 
     |████████████████████████████████| 3.4 MB 77.2 MB/s 
     |████████████████████████████████| 1.1 MB 74.6 MB/s 
     |████████████████████████████████| 77 kB 7.9 MB/s 
     |████████████████████████████████| 840 kB 42.4 MB/s 
     |████████████████████████████████| 1.1 MB 75.5 MB/s 
     |████████████████████████████████| 6.4 MB 59.3 MB/s 
     |████████████████████████████████| 1.2 MB 57.9 MB/s 
     |████████████████████████████████| 596 kB 67.8 MB/s 
     |████████████████████████████████| 25.3 MB 97.5 MB/s 
     |████████████████████████████████| 99 kB 12.3 MB/s 
     |████████████████████████████████| 47.7 MB 90.3 MB/s 
     |████████████████████████████████| 352 kB 88.2 MB/s 
     |████████████████████████████████| 462 kB 74.1 MB/s 
     |███████████

In [5]:
import numpy as np
import os

#from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

## Training 

In [6]:
train_image_dir = '/content/Final Img copy 2/train'
train_annotation_dir = '/content/Final Img copy 2/train anotation'
validate_image_dir = '/content/Final Img copy 2/valid'
validate_anaotation_dir = '/content/Final Img copy 2/valid anotation '

In [7]:

spec = model_spec.get('efficientdet_lite3')

In [8]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani", 7:"tshirt"})

In [9]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"Design Kurta", 2: "Plain kurta", 3: "Casual Shirt", 4:"Formal Shirt", 5: "Desgin sherwani", 6:"Plain sherwani",7:"tshirt"})

In [10]:
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=50, 
                               batch_size=5, 
                               train_whole_model=True, 
                               validation_data=validate_dir
                               )

Epoch 1/50
176/176 [==============================] - 151s 529ms/step - det_loss: 1.1169 - cls_loss: 0.8824 - box_loss: 0.0047 - reg_l2_loss: 0.0935 - loss: 1.2104 - learning_rate: 0.0071 - gradient_norm: 2.9153 - val_det_loss: 0.7617 - val_cls_loss: 0.6963 - val_box_loss: 0.0013 - val_reg_l2_loss: 0.0936 - val_loss: 0.8553
Epoch 2/50
176/176 [==============================] - 93s 529ms/step - det_loss: 0.5960 - cls_loss: 0.5262 - box_loss: 0.0014 - reg_l2_loss: 0.0936 - loss: 0.6897 - learning_rate: 0.0062 - gradient_norm: 3.1942 - val_det_loss: 0.6142 - val_cls_loss: 0.5235 - val_box_loss: 0.0018 - val_reg_l2_loss: 0.0937 - val_loss: 0.7080
Epoch 3/50
176/176 [==============================] - 94s 533ms/step - det_loss: 0.4703 - cls_loss: 0.4248 - box_loss: 9.1086e-04 - reg_l2_loss: 0.0938 - loss: 0.5641 - learning_rate: 0.0062 - gradient_norm: 3.3106 - val_det_loss: 0.5167 - val_cls_loss: 0.4809 - val_box_loss: 7.1659e-04 - val_reg_l2_loss: 0.0938 - val_loss: 0.6105
Epoch 4/50
176/1

In [11]:
model.history

AttributeError: ignored

## Evaluating the model 

In [12]:
model.evaluate(validate_dir,batch_size=5)

49/49 [==============================] - 17s 237ms/step



{'AP': 0.90297824,
 'AP50': 0.94939154,
 'AP75': 0.936893,
 'AP_/Casual Shirt': 0.9286085,
 'AP_/Desgin sherwani': 0.8649864,
 'AP_/Design Kurta': 0.8403961,
 'AP_/Formal Shirt': 0.94987535,
 'AP_/Plain kurta': 0.91531277,
 'AP_/Plain sherwani': 0.90719414,
 'AP_/tshirt': 0.9144746,
 'APl': 0.9031828,
 'APm': -1.0,
 'APs': -1.0,
 'ARl': 0.94684064,
 'ARm': -1.0,
 'ARmax1': 0.9308229,
 'ARmax10': 0.9452167,
 'ARmax100': 0.94684064,
 'ARs': -1.0}

## Saving Model 

In [13]:
model.export(export_dir='/content/')